# Doing Data Science: chapter 2 - Housing Dataset

Python code for the exercise on the RealDirect study about the housing dataset ("*improve the way people sell and buy houses*").

**Author**: Damien Garaud

**Project on Github**: https://github.com/garaud/doing_pydata_science

## Getting Data 

Clone the official Github project at https://github.com/oreillymedia/doing_data_science and unzip the `dds_datasets.zip` file. You'll find a new ZIP file named `dds_chapter2_rollingsales.zip`. Then, you'll get five XLS files:

- `rollingsales_bronx.xls`
- `rollingsales_brooklyn.xls`
- `rollingsales_manhattan.xls`
- `rollingsales_queens.xls`
- `rollingsales_statenisland.xls`

## Modules

In [1]:
import pandas as pd

## Reading Data

You continue to use pandas for XLS files reading. Suppose you have the `rollingsales_AREA.xls` files in the `data` directory.

The relevant row number to extract header is the row no.5 (note: row spreadsheet begins to 1).

In [2]:
brooklyn = pd.read_excel("./data/rollingsales_brooklyn.xls", header=4)

In [4]:
brooklyn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23373 entries, 0 to 23372
Data columns (total 21 columns):
BOROUGH                           23373 non-null int64
NEIGHBORHOOD                      23373 non-null object
BUILDING CLASS CATEGORY           23373 non-null object
TAX CLASS AT PRESENT              23373 non-null object
BLOCK                             23373 non-null int64
LOT                               23373 non-null int64
EASE-MENT                         23373 non-null object
BUILDING CLASS AT PRESENT         23373 non-null object
ADDRESS                           23373 non-null object
APART
MENT
NUMBER                 23373 non-null object
ZIP CODE                          23373 non-null int64
RESIDENTIAL UNITS                 23373 non-null int64
COMMERCIAL UNITS                  23373 non-null int64
TOTAL UNITS                       23373 non-null int64
LAND SQUARE FEET                  23373 non-null int64
GROSS SQUARE FEET                 23373 non-null int64
YEAR

21 columns and several different data

## Load and Clean up

Quoting: *First challenge: load in and clean up the data. Next, conduct
exploratory data analysis in order to find out where there are
outliers or missing values, decide how you will treat them, make
sure the dates are formatted correctly, make sure values you
think are numerical are being treated as such, etc.*